Looks at the same schema from different points in time, and then compares the schemata. 

This can't work properly in the current set-up. There isn't a 'this existed at this point'/ stopped existing.  
You would need to make the assumption that everything existed at a given point and only added stuff was different.  
This also hurts significantly from SQLite not having SProcs - what would be a table valued function is now limited to use only in this script.

TODO: implement 'is deleted' on running in a new scrape, but that's complicated by partial scrapes.

<span style="color: var(--vscode-foreground);">- does it have the same tables<br></span><span style="color: var(--vscode-foreground);">- does it have the same attributes on those tables<br></span><span style="color: var(--vscode-foreground);">- do the attributes have the same definitions</span>

NOT IMPLEMENTED YET  
\- are the FKs the same  
\- are the Indexes the same

In [19]:
import json
import os
import sqlite3
import csv

from pathlib import Path
from datetime import datetime

conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "../export_output_files/TXT/"


In [20]:
def fetch_data_as_dict(cursor, query, params):
  try:
    cursor.execute(query, params)
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    data_dict_list = [dict(zip(columns, row)) for row in results]
    return data_dict_list
  except sqlite3.Error as e:
    print(f"Error: {e}")
    return []

In [21]:
# the target schema, identified by server/ db/ schema
server_name = ''
database_name = ''
schema_name = 'dbo'

# Date/Time strings in the format 'YYYY-MM-DD' or 'YYYY-MM-DD HH24:MI:SS' or up to six decimal places of seconds
# Remember when doing day-dates, the cut-off is midnight of the start of the day, so updates on that day aren't covered.
apple_point_in_time = '2024-09-12'
orange_point_in_time = '2024-09-01'



In [22]:
# Get table diff
query = '''
-- Table names in a schema
WITH
Apple_Dataverse_Structure AS (
  SELECT
    ps.PhysicalStructureHashKey
    , sat."SERVER_NAME"
    , sat."DATABASE_NAME"
    , sat."SCHEMA_NAME"
    , sat."Logical Name" AS TABLE_NAME
    , eu."CountAttributes" AS COLUMN_COUNT
  FROM
    rv_h_PhysicalStructure ps
    INNER JOIN rv_s_PhysicalStructure_XRMMetadata AS sat ON (
      ps.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalStructure_XRMMetadata AS z
        WHERE
          z.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
          AND z.LoadDate <= ?
      )
    )
    LEFT OUTER JOIN rv_s_PhysicalStructure_XRMEntityUsage AS eu ON (
      ps.PhysicalStructureHashKey = eu.PhysicalStructureHashKey
      AND eu.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalStructure_XRMEntityUsage AS z
        WHERE
          z.PhysicalStructureHashKey = eu.PhysicalStructureHashKey
          AND z.LoadDate <= ?
      )
    )
)
, Apple_SQLServer_Structure AS (
  SELECT
    ps.PhysicalStructureHashKey
    , SERVER_NAME
    , "DATABASE_NAME"
    , SCHEMA_NAME
    , TABLE_NAME
    , COLUMN_COUNT
  FROM
    rv_h_PhysicalStructure ps
    INNER JOIN rv_s_PhysicalStructure_SqlServerScrape AS sat ON (
      ps.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalStructure_SqlServerScrape AS z
        WHERE
          z.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
          AND z.LoadDate <= ?
      )
    )
)
, Apple_Structure AS (
  SELECT * FROM Apple_Dataverse_Structure
  UNION
  SELECT * FROM Apple_SQLServer_Structure
)
, APPLES AS (
  SELECT DISTINCT
    'APPLE' AS SOURCE
    , TABLE_NAME
    , COLUMN_COUNT
  FROM
    Apple_Structure
  WHERE
    SERVER_NAME = ?
    AND DATABASE_NAME = ?
    AND SCHEMA_NAME = ?
)
, Orange_Dataverse_Structure AS (
  SELECT
    ps.PhysicalStructureHashKey
    , sat."SERVER_NAME"
    , sat."DATABASE_NAME"
    , sat."SCHEMA_NAME"
    , sat."Logical Name" AS TABLE_NAME
    , eu."CountAttributes" AS COLUMN_COUNT
  FROM
    rv_h_PhysicalStructure ps
    INNER JOIN rv_s_PhysicalStructure_XRMMetadata AS sat ON (
      ps.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalStructure_XRMMetadata AS z
        WHERE
          z.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
          AND z.LoadDate <= ?
      )
    )
    LEFT OUTER JOIN rv_s_PhysicalStructure_XRMEntityUsage AS eu ON (
      ps.PhysicalStructureHashKey = eu.PhysicalStructureHashKey
      AND eu.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalStructure_XRMEntityUsage AS z
        WHERE
          z.PhysicalStructureHashKey = eu.PhysicalStructureHashKey
          AND z.LoadDate <= ?
      )
    )
)
, Orange_SQLServer_Structure AS (
  SELECT
    ps.PhysicalStructureHashKey
    , SERVER_NAME
    , "DATABASE_NAME"
    , SCHEMA_NAME
    , TABLE_NAME
    , COLUMN_COUNT
  FROM
    rv_h_PhysicalStructure ps
    INNER JOIN rv_s_PhysicalStructure_SqlServerScrape AS sat ON (
      ps.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalStructure_SqlServerScrape AS z
        WHERE
          z.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
          AND z.LoadDate <= ?
      )
    )
)
, Orange_Structure AS (
  SELECT * FROM Orange_Dataverse_Structure
  UNION
  SELECT * FROM Orange_SQLServer_Structure
)
, ORANGES AS (
  SELECT DISTINCT
    'ORANGE' AS SOURCE
    , TABLE_NAME
    , COLUMN_COUNT
  FROM
    Orange_Structure
  WHERE
    SERVER_NAME = ?
    AND DATABASE_NAME = ?
    AND SCHEMA_NAME = ?
)
-- SQLite does not support full outer joins :(
-- have to do this in three stages.
, A_not_O AS (
  SELECT
    a.SOURCE AS In_Apples
    , o.SOURCE AS In_Oranges
    , a.TABLE_NAME
    , NULL AS IS_COLUMN_COUNT_DIFFERENT
  FROM
    APPLES a
    LEFT JOIN ORANGES o ON (a.TABLE_NAME = o.TABLE_NAME)
  WHERE
    o.SOURCE IS NULL -- in Apples, not in Oranges
)
, O_not_A AS (
  SELECT
    a.SOURCE AS In_Apples
    , o.SOURCE AS In_Oranges
    , o.TABLE_NAME
    , NULL AS IS_COLUMN_COUNT_DIFFERENT
  FROM
    ORANGES o
    LEFT JOIN APPLES a ON (a.TABLE_NAME = o.TABLE_NAME)
  WHERE
    a.SOURCE IS NULL -- in Oranges, not in Apples
)
, A_and_O AS (
  SELECT
    a.SOURCE AS In_Apples
    , o.SOURCE AS In_Oranges
    , a.TABLE_NAME
    , CASE
      WHEN (a.COLUMN_COUNT IS NULL AND o.COLUMN_COUNT IS NULL)
      THEN 0
      WHEN a.COLUMN_COUNT = o.COLUMN_COUNT
      THEN 0
      ELSE 1
    END AS IS_COLUMN_COUNT_DIFFERENT
  FROM
    APPLES a
    INNER JOIN ORANGES o ON (a.TABLE_NAME = o.TABLE_NAME)
)
, ONION AS (
  SELECT
    In_Apples
    , In_Oranges
    , TABLE_NAME
    , IS_COLUMN_COUNT_DIFFERENT
  FROM
    A_and_O

  UNION

  SELECT
    In_Apples
    , In_Oranges
    , TABLE_NAME
    , IS_COLUMN_COUNT_DIFFERENT
  FROM
    A_not_O

  UNION

  SELECT
    In_Apples
    , In_Oranges
    , TABLE_NAME
    , IS_COLUMN_COUNT_DIFFERENT
  FROM
    O_not_A
)
SELECT
  In_Apples
  , In_Oranges
  , TABLE_NAME
  , IS_COLUMN_COUNT_DIFFERENT
FROM
  ONION
ORDER BY
  TABLE_NAME
;

'''
params = [
  apple_point_in_time
  , apple_point_in_time
  , apple_point_in_time
  , server_name
  , database_name
  , schema_name
  , orange_point_in_time
  , orange_point_in_time
  , orange_point_in_time
  , server_name
  , database_name
  , schema_name
]
table_dict_list = fetch_data_as_dict(cur, query, params)

#for row_dict in table_dict_list:
#  print(row_dict)


In [23]:
output_filename = "schema_compare_tables_" + time_string + ".csv"

# Get the keys from the first dictionary as the header
header = table_dict_list[0].keys()

# Write to CSV file
with open(os.path.join(output_path, output_filename), 'w', newline='') as f:
  writer = csv.DictWriter(f, fieldnames=header)
  writer.writeheader()
  writer.writerows(table_dict_list)


In [24]:
# Get column diff
query = '''
-- Column names and datatypes in a schema
WITH
Apple_Dataverse_Structure AS (
  SELECT
    ps.PhysicalAttributeHashKey
   
    , sat."SERVER_NAME"
    , sat."DATABASE_NAME"
    , sat."SCHEMA_NAME"
    , sat."entity_logical_name" AS TABLE_NAME
    , sat."Logical Name" AS COLUMN_NAME

    , NULL AS IS_NULLABLE
    , sat."Attribute Type" AS DATA_TYPE
    , NULL AS CHARACTER_MAXIMUM_LENGTH
    , NULL AS NUMERIC_PRECISION
    , NULL AS NUMERIC_SCALE
  FROM
    rv_h_PhysicalAttribute ps
    INNER JOIN rv_s_PhysicalAttribute_XRMMetadata AS sat ON (
      ps.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalAttribute_XRMMetadata AS z
        WHERE
          z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
          AND z.LoadDate <= ?
      )
    )
)
, Apple_SQLServer_Structure AS (
SELECT
  ps.PhysicalAttributeHashKey
 
  , sat.SERVER_NAME
  , sat."DATABASE_NAME"
  , sat.SCHEMA_NAME
  , sat.TABLE_NAME
  , sat.COLUMN_NAME

  , sat.IS_NULLABLE
  , sat.DATA_TYPE
  , sat.CHARACTER_MAXIMUM_LENGTH
  , sat.NUMERIC_PRECISION
  , sat.NUMERIC_SCALE
 
FROM
  rv_h_PhysicalAttribute ps
  INNER JOIN rv_s_PhysicalAttribute_SqlServerScrape AS sat ON (
    ps.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalAttribute_SqlServerScrape AS z
      WHERE
        z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
        AND z.LoadDate <= ?
    )
  )
)
, Apple_Structure AS (
  SELECT * FROM Apple_Dataverse_Structure
  UNION
  SELECT * FROM Apple_SQLServer_Structure
)
, APPLES AS (
  SELECT DISTINCT
    'APPLE' AS SOURCE
    , TABLE_NAME
    , COLUMN_NAME

    , IS_NULLABLE
    , DATA_TYPE
    , CHARACTER_MAXIMUM_LENGTH
    , NUMERIC_PRECISION
    , NUMERIC_SCALE
  
    , IFNULL(IS_NULLABLE, 'NULL')
      || "|" || IFNULL(DATA_TYPE, 'NULL')
      || "|" || IFNULL(CHARACTER_MAXIMUM_LENGTH, 'NULL')
      || "|" || IFNULL(NUMERIC_PRECISION, 'NULL')
      || "|" || IFNULL(NUMERIC_SCALE, 'NULL')
    AS DATATYPE_HASH
  FROM
    Apple_Structure
  WHERE
    SERVER_NAME = ?
    AND DATABASE_NAME = ?
    AND SCHEMA_NAME = ?
    --AND TABLE_NAME = ''
)
, Orange_Dataverse_Structure AS (
  SELECT
    ps.PhysicalAttributeHashKey
   
    , sat."SERVER_NAME"
    , sat."DATABASE_NAME"
    , sat."SCHEMA_NAME"
    , sat."entity_logical_name" AS TABLE_NAME
    , sat."Logical Name" AS COLUMN_NAME

    , NULL AS IS_NULLABLE
    , sat."Attribute Type" AS DATA_TYPE
    , NULL AS CHARACTER_MAXIMUM_LENGTH
    , NULL AS NUMERIC_PRECISION
    , NULL AS NUMERIC_SCALE
  FROM
    rv_h_PhysicalAttribute ps
    INNER JOIN rv_s_PhysicalAttribute_XRMMetadata AS sat ON (
      ps.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalAttribute_XRMMetadata AS z
        WHERE
          z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
          AND z.LoadDate <= ?
      )
    )
)
, Orange_SQLServer_Structure AS (
SELECT
  ps.PhysicalAttributeHashKey
 
  , sat.SERVER_NAME
  , sat."DATABASE_NAME"
  , sat.SCHEMA_NAME
  , sat.TABLE_NAME
  , sat.COLUMN_NAME

  , sat.IS_NULLABLE
  , sat.DATA_TYPE
  , sat.CHARACTER_MAXIMUM_LENGTH
  , sat.NUMERIC_PRECISION
  , sat.NUMERIC_SCALE
 
FROM
  rv_h_PhysicalAttribute ps
  INNER JOIN rv_s_PhysicalAttribute_SqlServerScrape AS sat ON (
    ps.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalAttribute_SqlServerScrape AS z
      WHERE
        z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
        AND z.LoadDate <= ?
    )
  )
)
, Orange_Structure AS (
  SELECT * FROM Orange_Dataverse_Structure
  UNION
  SELECT * FROM Orange_SQLServer_Structure
)
, ORANGES AS (
  SELECT DISTINCT
    'ORANGE' AS SOURCE
    , TABLE_NAME
    , COLUMN_NAME

    , IS_NULLABLE
    , DATA_TYPE
    , CHARACTER_MAXIMUM_LENGTH
    , NUMERIC_PRECISION
    , NUMERIC_SCALE
  
    , IFNULL(IS_NULLABLE, 'NULL')
      || "|" || IFNULL(DATA_TYPE, 'NULL')
      || "|" || IFNULL(CHARACTER_MAXIMUM_LENGTH, 'NULL')
      || "|" || IFNULL(NUMERIC_PRECISION, 'NULL')
      || "|" || IFNULL(NUMERIC_SCALE, 'NULL')
    AS DATATYPE_HASH
  FROM
    Orange_Structure
  WHERE
    SERVER_NAME = ?
    AND DATABASE_NAME = ?
    AND SCHEMA_NAME = ?
    --AND TABLE_NAME = ''
)
-- SQLite does not support full outer joins :(
-- have to do this in three stages.
, A_not_O AS (
  SELECT
    a.SOURCE AS In_Apples
    , o.SOURCE AS In_Oranges
    , a.TABLE_NAME
    , a.COLUMN_NAME
    , NULL AS IS_DATATYPE_DIFFERENT
  FROM
    APPLES a
    LEFT JOIN ORANGES o ON (
      a.TABLE_NAME = o.TABLE_NAME
      AND a.COLUMN_NAME = o.COLUMN_NAME
    )
  WHERE
    o.SOURCE IS NULL -- in Apples, not in Oranges
)
, O_not_A AS (
  SELECT
    a.SOURCE AS In_Apples
    , o.SOURCE AS In_Oranges
    , o.TABLE_NAME
    , o.COLUMN_NAME
    , NULL AS IS_DATATYPE_DIFFERENT
  FROM
    ORANGES o
    LEFT JOIN APPLES a ON (
      a.TABLE_NAME = o.TABLE_NAME
      AND a.COLUMN_NAME = o.COLUMN_NAME
    )
  WHERE
    a.SOURCE IS NULL -- in Oranges, not in Apples
)
, A_and_O AS (
  SELECT
    a.SOURCE AS In_Apples
    , o.SOURCE AS In_Oranges
    , a.TABLE_NAME
    , a.COLUMN_NAME
    , CASE
      WHEN a.DATATYPE_HASH = o.DATATYPE_HASH
      THEN 0
      ELSE 1
    END AS IS_DATATYPE_DIFFERENT
  FROM
    APPLES a
    INNER JOIN ORANGES o ON (
      a.TABLE_NAME = o.TABLE_NAME
      AND a.COLUMN_NAME = o.COLUMN_NAME
    )
)
, ONION AS (
  SELECT
    In_Apples
    , In_Oranges
    , TABLE_NAME
    , COLUMN_NAME
    , IS_DATATYPE_DIFFERENT
  FROM
    A_and_O

  UNION

  SELECT
    In_Apples
    , In_Oranges
    , TABLE_NAME
    , COLUMN_NAME
    , IS_DATATYPE_DIFFERENT
  FROM
    A_not_O

  UNION

  SELECT
    In_Apples
    , In_Oranges
    , TABLE_NAME
    , COLUMN_NAME
    , IS_DATATYPE_DIFFERENT
  FROM
    O_not_A
)
SELECT
  In_Apples
  , In_Oranges
  , TABLE_NAME
  , COLUMN_NAME
  , IS_DATATYPE_DIFFERENT
FROM
  ONION
ORDER BY
  TABLE_NAME
  , COLUMN_NAME
;
'''
params = [
  apple_point_in_time
  , apple_point_in_time
  , server_name
  , database_name
  , schema_name
  , orange_point_in_time
  , orange_point_in_time
  , server_name
  , database_name
  , schema_name
]
field_dict_list = fetch_data_as_dict(cur, query, params)


#for row_dict in field_dict_list:
#  print(row_dict)


In [25]:
output_filename = "schema_compare_fields_" + time_string + ".csv"

# Get the keys from the first dictionary as the header
header = field_dict_list[0].keys()

# Write to CSV file
with open(os.path.join(output_path, output_filename), 'w', newline='') as f:
  writer = csv.DictWriter(f, fieldnames=header)
  writer.writeheader()
  writer.writerows(field_dict_list)
